In [1]:
!pip install augly[all]

In [2]:
import os
os.chdir('/content/drive/MyDrive/GT/DL/project/meme-challenge')
!pwd

/content/drive/MyDrive/GT/DL/project/meme-challenge


In [3]:
from PIL import Image

In [6]:
import pandas as pd

data_dir = './hateful_memes/'
dev_path = data_dir + "dev_unseen.jsonl"
test_path = data_dir + "test_unseen.jsonl"

data = pd.read_json(path_or_buf=test_path, lines=True)

In [9]:
import augly.image as imaugs
import augly.utils as utils
import augly.text as textaugs
import random
from tqdm import tqdm

def apply_aug(row):
    try:
        image = Image.open(f"./hateful_memes/img/{row['id']}.png")

        aug = imaugs.Compose(
            [
                imaugs.Saturation(factor=2.0),
                imaugs.Scale(factor=0.6),
                imaugs.OverlayEmoji(emoji_path="./heart_eyes.jpg", emoji_size=0.1, x_pos=random.random(), y_pos=random.random(), opacity=0.4),
                imaugs.RandomNoise(mean=0.0,var=0.005)
            ]
        )
        aug_img = aug(image)
        aug_img.save(f"./augmented/augmented_{row['id']}.png")

        input_text = row['text']
        text_aug = textaugs.simulate_typos(input_text)

        row['aug_text'] = text_aug
        row['aug_img'] = f"augmented_{row['id']}.png"

    except FileNotFoundError as e:
        pass

    return row

aug_data = data.apply(apply_aug,axis=1)

In [10]:
aug_data

,aug_img,aug_text,id,img,label,text
0,augmented_15740.png,when someone tell0s you how to bb!,15740,img/15740.png,1,when someone tells you how to bbq
1,augmented_38794.png,when they say white folks don' t kwno how to cook,38794,img/38794.png,1,when they say white folks don't know how to cook
2,augmented_60792.png,the original der-face,60792,img/60792.png,1,the original derp-face
3,augmented_71824.png,o!kay here you go! you piece $of shi!,71824,img/71824.png,1,okay here you go! you piece of shit!
4,NaN,NaN,4796,img/04796.png,1,xboxone farming 1619 simulator
...,...,...,...,...,...,...
1995,augmented_71352.png,fighting for gay rightls,71352,img/71352.png,0,fighting for gay rights
1996,NaN,NaN,2164,img/02164.png,0,that feeling when you finish your homework in ...
1997,NaN,NaN,3587,img/03587.png,0,the day that shook new york city
1998,augmented_47839.png,one of the first protptypes of the atgom bomb,47839,img/47839.png,0,one of the first prototypes of the atom bomb


In [8]:
import json
import pandas as pd

with open("aug_test_unseen.jsonl", "w") as f:
    f.write(aug_data.to_json(orient='records', lines=True, force_ascii=False).replace('\/', '/'))